In [6]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.documents import Document
from langchain_chroma import Chroma
from uuid import uuid4
import os

from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(
    base_url="http://172.16.5.60:3000",
    model="embeddinggemma:300m"
)

baseFilepath = "./Data/deps/pdfs"
# File processing

listedFiles = os.listdir(baseFilepath)
docPath = os.path.join(baseFilepath, listedFiles[1])

In [7]:
# Manual document parsing

loader = PyPDFLoader(docPath)
docs  = loader.load()
docs[2].page_content

'Summary of Last Lecture\n• The ideal gas equation is given by\n• The gas constant can be found from the specific heats\n• Superheated water vapour can be considered as an ideal gas at high \ntemperatures and relatively low pressures\n𝑅 = 𝐶𝑝 − 𝐶𝑣𝑅 =\nത𝑅\n𝑀 ;\n𝑝𝑉 = 𝑚𝑅𝑇; 𝑝ν = 𝑅𝑇; 𝑝ν =\nത𝑅𝑇\n𝑀 ; 𝑝 = 𝜌𝑅𝑇'

In [12]:
def loadPdf(pdf: PyPDFLoader):
    # Initing values
    docs = pdf.load() # Unpacking the pdf

    vector_store = Chroma(
        collection_name="pdf_collection",
        embedding_function=embeddings,
        persist_directory="./Data/Chroma",  # Where to save data locally, remove if not necessary
    )

    result_docs = []
    # Creating reliable documents
    for page in docs:
        pageLabel = page.metadata["page_label"]

        document = Document(
            page_content= "Page: " + pageLabel + "\n\n" + page.page_content,
            metadata={"page_label": page.metadata["page_label"], "source": page.metadata["source"]}
        )

        result_docs.append(document)

    # Creating id for vector store
    uuids = [str(uuid4()) for _ in range(len(result_docs))]

    vector_store.add_documents(documents=result_docs, ids=uuids)

In [13]:
pdf = PyPDFLoader(docPath)
loadPdf(pdf)

In [15]:
# querying based on database
vector_store = Chroma(
    collection_name="pdf_collection",
    embedding_function=embeddings,
    persist_directory="./Data/Chroma",  # Where to save data locally, remove if not necessary
)

results = vector_store.similarity_search(
    "Ideal gas"
    # k=2
)

results

[Document(id='56e6b506-2b11-41b3-854f-dc03f02a70a8', metadata={'page_label': '3', 'source': './Data/deps/pdfs/Week 9 - Conservation of Energy for Steady Flow Systems2.pdf'}, page_content='Page: 3\n\nSummary of Last Lecture\n• The ideal gas equation is given by\n• The gas constant can be found from the specific heats\n• Superheated water vapour can be considered as an ideal gas at high \ntemperatures and relatively low pressures\n𝑅 = 𝐶𝑝 − 𝐶𝑣𝑅 =\nത𝑅\n𝑀 ;\n𝑝𝑉 = 𝑚𝑅𝑇; 𝑝ν = 𝑅𝑇; 𝑝ν =\nത𝑅𝑇\n𝑀 ; 𝑝 = 𝜌𝑅𝑇'),
 Document(id='547435de-ec62-4f57-b609-47f91fb0c4d3', metadata={'page_label': '3', 'source': './Data/deps/pdfs/Week 9 - Conservation of Energy for Steady Flow Systems2.pdf'}, page_content='Page: 3\n\nSummary of Last Lecture\n• The ideal gas equation is given by\n• The gas constant can be found from the specific heats\n• Superheated water vapour can be considered as an ideal gas at high \ntemperatures and relatively low pressures\n𝑅 = 𝐶𝑝 − 𝐶𝑣𝑅 =\nത𝑅\n𝑀 ;\n𝑝𝑉 = 𝑚𝑅𝑇; 𝑝ν = 𝑅𝑇; 𝑝ν =\nത𝑅𝑇\n𝑀 ; 𝑝 = 𝜌